# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_hw import g_key

### Store Part I results into DataFrame

In [2]:
cities_info = pd.read_csv("../WeatherPy/cities_final.csv")

### Humidity Heatmap

In [3]:
gmaps.configure(api_key=g_key)
#figure_layout ={}

fig = gmaps.figure()

In [4]:
cities_info.dropna(inplace=True)

In [5]:
locations = cities_info[["Lat", "Lng"]]
humidity = cities_info["Humidity"]

heat_layer = gmaps.heatmap_layer(
    locations,
    weights = humidity ,
    dissipating = False,
    point_radius=3,
    max_intensity =100 
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
condition_1 = cities_info["Cloudiness"]<5
condition_2 = (cities_info["Max Temp"]>65) & (cities_info["Max Temp"]<80)
condition_3 = (cities_info["Wind Speed"]<5)
conditions_df = cities_info[condition_1 & condition_2 & condition_3].head(5)
conditions_df = conditions_df.dropna()
conditions_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
33,hilo,1.0,US,1.589204e+09,68.0,19.73,-155.09,69.80,4.70
61,huejuquilla el alto,0.0,MX,1.589205e+09,35.0,22.60,-103.87,65.66,1.34
224,tocopilla,4.0,CL,1.589205e+09,51.0,-22.09,-70.20,68.68,4.99
301,morgan city,1.0,US,1.589205e+09,72.0,29.70,-91.21,66.00,4.70
352,gorontalo,0.0,ID,1.589205e+09,85.0,0.54,123.06,77.52,2.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
conditions_df["Hotel Name"] = ""
conditions_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
33,hilo,1.0,US,1.589204e+09,68.0,19.73,-155.09,69.80,4.70,
61,huejuquilla el alto,0.0,MX,1.589205e+09,35.0,22.60,-103.87,65.66,1.34,
224,tocopilla,4.0,CL,1.589205e+09,51.0,-22.09,-70.20,68.68,4.99,
301,morgan city,1.0,US,1.589205e+09,72.0,29.70,-91.21,66.00,4.70,
352,gorontalo,0.0,ID,1.589205e+09,85.0,0.54,123.06,77.52,2.95,


In [8]:
for index, row in conditions_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    target_coordinates = f"{lat},{lng}"
    target_radius = 5000
    target_search = "hotel"
    target_type = "lodging"
    
    params = { 
    "location": target_coordinates,
    "radius": target_radius,
    "keyword": target_search,
    "type": target_type,
    "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        
        conditions_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except(KeyError, IndexError):
        print("Hotel not found. Skipping...")


Closest hotel to hilo is Grand Naniloa Hotel Hilo - a DoubleTree by Hilton.
Closest hotel to huejuquilla el alto is Hotel Huexuquillan.
Closest hotel to tocopilla is Hostal Paraiso.
Closest hotel to morgan city is Red Roof Inn Morgan City.
Closest hotel to gorontalo is Gorontalo Inn Hotel.


In [9]:
conditions_df.replace("", np.nan, inplace=True)
conditions_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
33,hilo,1.0,US,1.589204e+09,68.0,19.73,-155.09,69.80,4.70,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
61,huejuquilla el alto,0.0,MX,1.589205e+09,35.0,22.60,-103.87,65.66,1.34,Hotel Huexuquillan
224,tocopilla,4.0,CL,1.589205e+09,51.0,-22.09,-70.20,68.68,4.99,Hostal Paraiso
301,morgan city,1.0,US,1.589205e+09,72.0,29.70,-91.21,66.00,4.70,Red Roof Inn Morgan City
352,gorontalo,0.0,ID,1.589205e+09,85.0,0.54,123.06,77.52,2.95,Gorontalo Inn Hotel


In [10]:
lat = conditions_df["Lat"]
lng = conditions_df["Lng"]
lat_lng = list(zip(lat,lng ))
marker_locations = lat_lng
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)

In [11]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in conditions_df.iterrows()]
locations = conditions_df[["Lat", "Lng"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))